In [2]:
import pandas as pd
import numpy as np
import ast

In [3]:
new_df = pd.read_csv('../movies.csv')

In [4]:
new_df.shape

(7346, 5)

In [5]:
new_df.duplicated().sum()

0

In [6]:
# Convert string representation of lists into actual lists
new_df['tags'] = new_df['tags'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Check the first few 
new_df['tags'][0]

['After',
 'receiving',
 'an',
 'unexpected',
 'call',
 'from',
 'her',
 'wayfinding',
 'ancestors,',
 'Moana',
 'journeys',
 'alongside',
 'Maui',
 'and',
 'a',
 'new',
 'crew',
 'to',
 'the',
 'far',
 'seas',
 'of',
 'Oceania',
 'and',
 'into',
 'dangerous,',
 'long-lost',
 'waters',
 'for',
 'an',
 'adventure',
 'unlike',
 'anything',
 "she's",
 'ever',
 'faced.',
 'Animation',
 'Adventure',
 'Family',
 'Comedy',
 'sea',
 'ocean',
 'villain',
 'musical',
 'sequel',
 'duringcreditsstinger',
 'absurd',
 'AuliʻiCravalho',
 'DwayneJohnson',
 'HualālaiChung',
 'DavidG.DerrickJr.',
 'JasonHand',
 'DanaLedouxMiller']

In [7]:
new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))

In [8]:
print(new_df['title'][100])
print(new_df['tags'][100])

Pleasure
19 year old Linnéa leaves her small town in Sweden and heads for Los Angeles with the aim of becoming the world's next big porn star, but the road to her goal turns out to be bumpier than she imagined. Drama pornography femalefriendship pornstar femaleprotagonist losangeles california pornindustry pornactress womandirector intimate absurd disgusted SofiaKappel ZeldaMorrison TeeReel JessamynLand JasonM.Roberts MikeRoque ZacharyWalker KhayiClement EmilyMathason AdamAndersson Kentucky JeniferEllis StevenRodgers SarahGarrettson KatieScholz NinjaThyberg


In [9]:
new_df['tags'] = new_df['tags'].apply(lambda x: x.lower())

In [10]:
new_df['tags'][100]

"19 year old linnéa leaves her small town in sweden and heads for los angeles with the aim of becoming the world's next big porn star, but the road to her goal turns out to be bumpier than she imagined. drama pornography femalefriendship pornstar femaleprotagonist losangeles california pornindustry pornactress womandirector intimate absurd disgusted sofiakappel zeldamorrison teereel jessamynland jasonm.roberts mikeroque zacharywalker khayiclement emilymathason adamandersson kentucky jeniferellis stevenrodgers sarahgarrettson katiescholz ninjathyberg"

In [11]:
new_df.shape

(7346, 5)

In [12]:
import re
from nltk.stem import PorterStemmer

ps = PorterStemmer()

def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text).lower()
    return ' '.join([ps.stem(word) for word in text.split()])




In [13]:
new_df['tags'] = new_df['tags'].apply(clean_text)

In [14]:
new_df.shape

(7346, 5)

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english', token_pattern=r'\b\w+\b')

In [16]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [17]:
vectors[0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [18]:
from sklearn.metrics.pairwise import cosine_similarity

In [19]:
similarity = cosine_similarity(vectors)

In [20]:
similarity

array([[1.        , 0.05679618, 0.        , ..., 0.02400077, 0.02677398,
        0.02913583],
       [0.05679618, 1.        , 0.075     , ..., 0.08451543, 0.14142136,
        0.07694838],
       [0.        , 0.075     , 1.        , ..., 0.02112886, 0.04714045,
        0.02564946],
       ...,
       [0.02400077, 0.08451543, 0.02112886, ..., 1.        , 0.03984095,
        0.02167775],
       [0.02677398, 0.14142136, 0.04714045, ..., 0.03984095, 1.        ,
        0.07254763],
       [0.02913583, 0.07694838, 0.02564946, ..., 0.02167775, 0.07254763,
        1.        ]])

In [21]:
similarity.shape

(7346, 7346)

In [22]:
new_df[new_df['title'] == 'Avatar'].index[0]

166

In [23]:
new_df[new_df['title'] == 'Batman Begins'].index[0]

3894

In [24]:
sorted(list(enumerate(similarity[0])), reverse=True, key=lambda x : x[1])[1:6]

[(3952, 0.3592106040535498),
 (1377, 0.35560035560053344),
 (1721, 0.33256439728421594),
 (864, 0.32128773156099955),
 (2117, 0.31108550841912763)]

In [25]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distance = similarity[movie_index]
    sort_list = sorted(list(enumerate(distance)), reverse=True, key=lambda x: x[1])

    movies_list = []  # Use a list instead of a set

    for i in sort_list:
        if new_df.iloc[i[0]].title == movie: 
            continue  # Skip the same movie

        if new_df.iloc[i[0]].vote_average >= 7 and len(movies_list) < 6:  # Stop at 6 movies
            movies_list.append(new_df.iloc[i[0]].title)

    return movies_list  # Return the list instead of printing



In [26]:
# Example usage:
recommend("Batman Begins")


['Batman: The Dark Knight Returns, Part 1',
 'The Dark Knight',
 'The Raid 2',
 'Batman',
 'Batman: Under the Red Hood',
 'Batman: Mask of the Phantasm']

In [27]:
import pickle

In [28]:
pickle.dump(new_df, open('../movies.pkl', 'wb'))

In [29]:
pickle.dump(similarity, open('../pythonProject/similarity.pkl', 'wb'))

In [49]:
new_df.sample()

,Unnamed: 0,id,title,tags,vote_average
2951,2951,12572,"New York, I Love You",new york i love you delv into the intim live o...,5.942


In [56]:
movie = 'New York, I Love You'
current_movie_vote = float(new_df[new_df['title'] == movie].vote_average)
current_movie_vote

5.942